![imagenes](logo.png)

# Pruebas de normalidad

Frecuentemente nos encontramos frente al problema de reconocer si una muestra proviene de una población gaussiana.

Aunque ya hemos visto ajuste de distribuciones, para el caso de gaussianas tenemos más métodos para establecer, con cierta probabilidad, si una muestra proviene realmente de una distribución normal. Estos métodos están basados en pruebas de hipótesis, donde la hipótesis nula tiene la forma "la población es gaussiana", en tanto que la alternativa es "la población no es gaussiana".

De esta manera, a cada una de las pruebas se le asigna un estadístico de contraste con el cual se calcula un $p$-valor, de modo que podemos escribirlas así:

$$\left\{\begin{array}{l}H_0:\mbox{ la población es gaussiana}\\H_1:\mbox{ la población no es gaussiana}\end{array}\right.$$

con regla de decisión:

$p$-valor|Decisión|Significado
:--|:--|:--
Pequeño|Rechazar $H_0$|Hay buena probabilidad de que **NO es gaussiana**
Grande|Rechazar $H_1$|Hay buena probabilidad de que **SÍ es gaussiana**

En este capítulo estudiaremos cuatro contrastes que nos ayudan para este fin:

1. **Prueba de Shapiro-Wilk**: Una de las pruebas más populares y potentes para detectar desviaciones de la normalidad, especialmente en muestras pequeñas.
2. **Prueba de Kolmogorov-Smirnov**: Una prueba basada en la comparación entre la distribución empírica y la distribución normal teórica. 
3. **Prueba de Anderson-Darling**: Una extensión del contraste de Kolmogorov-Smirnov que da más peso a las colas de la distribución.
4. **Prueba de Jarque-Bera**: Evalúa la normalidad basándose en los momentos estadísticos de asimetría y curtosis.

## Contexto e importancia de las pruebas de normalidad

En estadística, muchas pruebas y modelos (como ANOVA, regresión lineal, t de Student, entre otros) requieren que los datos sigan una distribución normal. Por ello, las pruebas de normalidad son un paso esencial para garantizar la validez de los resultados.

Es importante considerar también que:

- **Tamaños de muestra pequeños**: Las pruebas pueden tener poca potencia, y un histograma o gráfico Q-Q puede complementar la evaluación.
- **Tamaños de muestra grandes**: Las pruebas tienden a detectar pequeñas desviaciones que pueden no ser prácticamente significativas.

A continuación, describiremos cada prueba en detalle y veremos cómo implementarlas usando Python.

Para esto, utilizaremos las siguientes muestras:

```python
import numpy as np
import scipy.stats as stats, lognorm
import random


#generada con N(mu=3.5,sigma=2)
small_gauss = [2.9267007, 2.5763093, 4.9931801, 0.6564296, 1.4377333, 7.6412183, 2.9204735] 

#generada con t(3)
big_t = [-0.577103929579228, -0.0669625949987604, 0.123572935953355, -0.524985500797433, -1.23249669279686, 0.509597230395874, -0.729559305649031, -0.41684441016622, 1.28155478163868, 0.924508782035897, 0.827405247774813, 1.59785194962189, -1.47879497630707, -1.26201626124022, -0.0593983026205043, -0.178873361732746, 0.801185847793428, 0.333473064862654, 1.25186288055626, 2.35949695172828, -0.633493106081742, -1.05713142223298, 0.0212461334293823, 0.466063027431909, 0.0762121526958427, -0.843837287109611, -0.104022595760381, 5.78550093074697, 0.709799846598426, -0.0897824055310009, -0.999402655342385, 0.337761665033848, -0.0306307006025367, 1.47728344947859, -0.176164802725808, 0.690341335235668, -0.292183630229324, -0.844902899428558, -3.49551302890857, 1.43006662844371, 1.24850000914668, -0.180820066444685, -0.573485189819109, 0.349757398842014, -2.09754115696913, -0.352572352149588, -0.509125036161415, 0.712742491824159, 0.519051722042105, -3.00737218678664]

#generada con N(mu=5,sigma=1)
random.seed(2024)
big_gauss = stats.norm(scale=1, loc=5).rvs(1000)

# Tomado de https://webspace.ship.edu/pgmarr/Geo441/Lectures/Lec%205%20-%20Normality%20Testing.pdf
densidades_mexico = {
    'Region': [
        'Ajuno', 'Angahuan', 'Arantepacua', 'Aranza', 'Charapan', 'Cheran',
        'Cocucho', 'Comachuen', 'Corupo', 'Ihuatzio', 'Janitzio', 'Jaracuaro',
        'Nahuatzen', 'Nurio', 'Paracho', 'Patzcuaro', 'Pichataro',
        'Pomacuaran', 'Quinceo', 'Quiroga', 'San Felipe', 'San Lorenzo',
        'Sevina', 'Tingambato', 'Turicuaro', 'Tzintzuntzan', 'Urapicho'
    ],
    'Population_Density': [
        5.11, 5.15, 5.00, 4.13, 5.10, 5.22, 5.04, 5.25, 4.53, 5.74, 6.63, 5.73,
        4.77, 6.06, 4.82, 4.98, 5.36, 4.96, 5.94, 5.01, 4.10, 4.69, 4.97, 5.01,
        6.19, 4.67, 6.30
    ]
}

# Generada con lognorm con media e y desviación 0.5
np.random.seed(1)
mi_lognorm = lognorm.rvs(s=.5, scale=math.exp(1), size=1000)

## 1. Prueba de Shapiro-Wilk

La prueba de Shapiro-Wilk evalúa si una muestra sigue una distribución normal al calcular un estadístico $W$ basado en el orden de los datos. Funiona bien para tamaños tan bajos como muestras de tamaño 3.

### Implementación en Python

Usaremos la función `shapiro` de la biblioteca `scipy.stats`.

```python
from scipy.stats import shapiro

# Prueba de Shapiro-Wilk
stat, p_value = shapiro(data)

print(f"Estadístico W: {stat:.4f}")
print(f"p-valor: {p_value:.4f}")

# Regla de decisión
if p_value < 0.05:
    print("Rechazamos H0: La muestra no sigue una distribución normal")
else:
    print("No podemos rechazar H0: La muestra sigue una distribución normal")
```

## 2. Prueba de Kolmogorov-Smirnov

Esta prueba compara la distribución empírica de los datos con la distribución normal teórica.

### Implementación en Python
Usaremos la función ``kstest`` de ``scipy.stats``.

```python
from scipy.stats import kstest, norm

# Prueba de Kolmogorov-Smirnov
from statistics import mean, stdev
stat, p_value = kstest(data, 'norm', args=(mean(data), stdev(data)))

print(f"Estadístico KS: {stat:.4f}")
print(f"p-valor: {p_value:.4f}")
```

## 3. Prueba de Anderson-Darling
La prueba de Anderson-Darling ajusta el estadístico de Kolmogorov-Smirnov para dar más peso a las colas.

### Implementación en Python
Usaremos la función ``anderson`` de ``scipy.stats``.

```python
from scipy.stats import anderson

# Prueba de Anderson-Darling
result = anderson(data, dist='norm')

print(f"Estadístico A: {result.statistic:.4f}")
for i, sig in enumerate(result.significance_level):
    print(f"Nivel de significancia {sig}%: Valor crítico = {result.critical_values[i]:.4f}")


```

## 4. Prueba de Jarque-Bera
Esta prueba evalúa la normalidad mediante los coeficientes de asimetría y curtosis.

### Implementación en Python
Usaremos la función ``jarque_bera`` de ``scipy.stats``.

```python
from scipy.stats import jarque_bera

# Prueba de Jarque-Bera
stat, p_value = jarque_bera(data)

print(f"Estadístico JB: {stat:.4f}")
print(f"p-valor: {p_value:.4f}")

```

## Algunas consideraciones 

Las pruebas de normalidad deben interpretarse en contexto, especialmente considerando el tamaño de la muestra.

- Las representaciones gráficas, como histogramas o diagramas Q-Q, son una herramienta valiosa complementaria.
- Aunque estas pruebas nos ayudan a evaluar la normalidad, la robustez de muchos modelos estadísticos permite cierto grado de desviación sin afectar significativamente los resultados.

## Resumen

A continuación mostramos una tabla resumen de estos contrastes de normalidad. **Recuerda que para utilizarlos en Python requieres la biblioteca ``scipy.stats`` y/o ``statsmodels``**.

Prueba|Abreviatura|Tamaño de<br> muestra|Desventaja|Estadístico|**Python**
:--|:--:|:--:|:--|:--:|:--
Shapiro-Wilk|SW|$n \leq 5000$|Sensible a valores repetidos y colas|W|``shapiro()``
Kolmogorov-Smirnov|KS|$n \leq 1000$|Requiere especificar parámetros|D|``kstest()``
Anderson-Darling|AD|$n \leq 5000$|Menos eficiente en muestras grandes|A|``anderson()``
Jarque-Bera|JB|$n \geq 20$|Menos sensible a desviaciones pequeñas|JB|``jarque_bera()``

En el script puedes encontrar nuestra función genérica para realizar este tipo de pruebas.

## Consideraciones finales

1. **Tamaño de muestra:**
   - Las pruebas como **Shapiro-Wilk** y **Anderson-Darling** son ideales para tamaños de muestra pequeños a moderados (hasta 5000 observaciones).
   - **Kolmogorov-Smirnov** se recomienda para tamaños pequeños a medianos (hasta 1000), ya que su eficiencia decrece con muestras grandes.
   - **Jarque-Bera** es más confiable en tamaños de muestra grandes (n ≥ 20) debido a su dependencia de estadísticos como la asimetría y la curtosis.

2. **Repeticiones y valores atípicos:**
   - Las pruebas como **Shapiro-Wilk** pueden verse afectadas por valores repetidos o extremos, lo que puede comprometer su capacidad para detectar normalidad.
   - Gráficos complementarios como histogramas o diagramas Q-Q son útiles para identificar patrones visuales antes de realizar pruebas.

3. **Colas de la distribución:**
   - **Anderson-Darling** da mayor peso a las colas, lo que la hace especialmente útil si se sospechan problemas en los extremos de la distribución.

4. **Pruebas específicas vs. generales:**
   - **Shapiro-Wilk** y **Anderson-Darling** están diseñadas exclusivamente para evaluar normalidad.
   - **Jarque-Bera** es una prueba "ómnibus", útil si deseas evaluar otros momentos estadísticos (asimetría y curtosis) junto con la normalidad.

5. **Efecto del tamaño de muestra:**
   - En muestras pequeñas, las pruebas de normalidad pueden tener poca potencia y no detectar desviaciones significativas.
   - En muestras grandes, incluso pequeñas desviaciones de la normalidad pueden resultar en un rechazo de $H_0$, aunque estas desviaciones no sean prácticas ni relevantes para muchos análisis.

6. **Apoyo visual:**
   - Complementar las pruebas estadísticas con visualizaciones como histogramas y diagramas Q-Q es una buena práctica para entender mejor los datos y confirmar los resultados.

En conclusión, la elección de la prueba depende del tamaño de la muestra, las características de los datos y la naturaleza del análisis. Las representaciones gráficas son un valioso complemento a estas pruebas formales.

## Comentario final

Además de todo lo anteriormente dicho, también es importante mencionar que en muchas ocasiones es necesario realizar una transformación previa a los datos para obtener gaussianidad. Las transormaciones más usuales son logaritmo, raíz cuadrada y raíz cúbica:

```python
import numpy as np

datos_log = np.log(datos)
datos_sqrt = np.sqrt(datos)
datos_sqrt3 = np.cbrt(datos)
```